In [1]:
import re
import os
from progressbar import ProgressBar
import numpy as np
from sklearn.metrics import precision_recall_fscore_support,f1_score
from collections import Counter as c
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix

In [6]:
class topnclass:
    def __init__(self):
        with open("/home/student1/parsefull.csv") as f:
            lin=f.readlines()
        self.con = [x.rstrip('\n') for x in lin]  ####### stripping newlines.........
        self.con =[re.split(':\d+',x) for x in self.con]
    def toplabels(self,n):
        con_new=[x[0]for x in self.con]
        con_new=[list(map(int,re.split('\s',x.replace(",",''))[:-1]))for x in con_new]
        flat_con_new = [item for sublist in con_new for item in sublist]
        label_counts=c(flat_con_new)
        label_top=[x[0]for x in label_counts.most_common(n)]
        return label_top
    def createdatatop(self,n):
        pbar=ProgressBar()
        label_list=self.toplabels(n)
        dict_for_labels_bools={x:0for x in range(445730)}
        for el in label_list:           #########using dict to create the data fastly
            dict_for_labels_bools[el]=1
        self.label_list_all=label_list
        ############# discard only labels not data
        tmp=[]
        y=[]###########target class
        X=[]#############documents
        with open('top'+str(n)+'classdata.txt','w') as f:
            for j in pbar(range(len(self.con))):
                tmp=re.split(', ',self.con[j][0])
                count=0
                tmp1=[]
                for x in tmp[:-1]:
                    if dict_for_labels_bools[int(x)]==1:
                        count = count+1
                        f.write('__label__')
                        tmp1.append(int(x))
                        f.write(str(x)+' ')
                tmp=tmp[-1].split()
                if dict_for_labels_bools[int(tmp[0])]==1:
                    count=count+1
                    f.write('__label__')
                    tmp1.append(int(tmp[0]))
                    f.write(str(tmp[0])+' ')
                if tmp1:
                    y.append(tmp1)
                if count>0:
                    f.write('w'+str(tmp[1])+' ')
                    tmp2=[]
                    for k in self.con[j][1:-1]:
                        f.write('w'+str(k).replace(' ','')+' ')
                        tmp2.append('w'+str(k).replace(' ',''))
                    X.append(" ".join(tmp2))
                    f.write('\n')
        return X,y
    def fastontop(self,n):
        pbar=ProgressBar()
        X,y=self.createdatatop(n)
        X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.30, random_state=30,shuffle=True)
        with open('top'+str(n)+'classtrain.txt','w') as f:
            for i in range(len(X_train)):
                for j in range(len(y_train[i])):
                    f.write("__label__")
                    f.write(str(y_train[i][j])+" ")
                f.write(X_train[i])
                f.write("\n")
        with open('top'+str(n)+'classtest.txt','w') as f:
            for i in range(len(X_test)):
                f.write(X_test[i])
                f.write("\n")
        cmd1="fasttext supervised  -dim 200 -epoch 100 -lr 0.25 -loss hs -input top"+str(n)+"classtrain.txt -output top"+str(n)+"classmodel"
        mod="top"+str(n)+"classmodel.bin "
        tes="top"+str(n)+"classtest.txt "
        res="top"+str(n)+"classresult.txt"
        y_train_labellength=[len(x) for x in y_train]
        Average_labels_per_document = sum(y_train_labellength)/len(y_train_labellength)
        print('Average_labels_per_document in top '+str(n)+' class data is',Average_labels_per_document)
        label_predict_count=str(round(Average_labels_per_document))
        print('So,Predicting ',label_predict_count,'labels using fasttext')
        cmd2="fasttext predict "+mod+ tes+label_predict_count+' > '+ res
        os.system(cmd1)
        os.system(cmd2)
        f = open(res, 'r')###############reading predicted values
        lines =[re.split('__label__', line) for line in f.readlines()]
        f.close()
        lines=[x[1:]for x in lines]
        lines=[list(map(int, x)) if isinstance(x,list) else [int(x)] for x in lines]
        y_true_row=[]###############################################generating sparse matrices of predicted and true labels for test docs
        y_true_col=[]
        y_true_data=[]
        y_pred_row=[]
        y_pred_col=[]
        y_pred_data=[]
        dict_for_label_indexes={}#############################using dict to get indexes of labels fastly
        counter_for_index_for_labels=0
        for x in self.label_list_all:
            dict_for_label_indexes[x]=counter_for_index_for_labels
            counter_for_index_for_labels=counter_for_index_for_labels+1
        for x in pbar(range(len(y_test))):
            t1=lines[x]
            t2=y_test[x]
            for k in range(len(t1)):
                y_pred_row.append(x)
                index1=dict_for_label_indexes[t1[k]]
                y_pred_col.append(index1)
                y_pred_data.append(1)
            for z in range(len(t2)):
                y_true_row.append(x)
                index2=dict_for_label_indexes[t2[z]]
                y_true_col.append(index2)
                y_true_data.append(1)
        y_pred_sparse=csr_matrix((y_pred_data, (y_pred_row, y_pred_col)),shape=(len(y_test),n))
        y_true_sparse=csr_matrix((y_true_data, (y_true_row, y_true_col)),shape=(len(y_test),n))
        print("For top "+str(n)+" class (precision,recall,fscore) using macro average:",precision_recall_fscore_support(y_true_sparse,y_pred_sparse, average='macro'))


In [7]:
obj=topnclass()

In [8]:
obj.fastontop(10)

100% (2365436 of 2365436) |###############| Elapsed Time: 0:00:40 Time: 0:00:40


Average_labels_per_document in top 10 class data is 1.2040520887536859
So,Predicting  1 labels using fasttext


100% (128485 of 128485) |#################| Elapsed Time: 0:00:00 Time: 0:00:00


For top 10 class (precision,recall,fscore) using macro average: (0.772653816146385, 0.4538973999545054, 0.5446300139373246, None)


In [9]:
obj.fastontop(100)

100% (2365436 of 2365436) |###############| Elapsed Time: 0:00:46 Time: 0:00:46


Average_labels_per_document in top 100 class data is 1.4025902858246426
So,Predicting  1 labels using fasttext


100% (177532 of 177532) |#################| Elapsed Time: 0:00:00 Time: 0:00:00


For top 100 class (precision,recall,fscore) using macro average: (0.5054905503878989, 0.29523742412066367, 0.3544216307196997, None)


In [10]:
obj.fastontop(1000)

100% (2365436 of 2365436) |###############| Elapsed Time: 0:00:59 Time: 0:00:59


Average_labels_per_document in top 1000 class data is 1.9115519552849054
So,Predicting  2 labels using fasttext


100% (257018 of 257018) |#################| Elapsed Time: 0:00:01 Time: 0:00:01


For top 1000 class (precision,recall,fscore) using macro average: (0.34433744845531866, 0.3388833539139551, 0.3225272593098418, None)


In [11]:
obj.fastontop(10000)

100% (2365436 of 2365436) |###############| Elapsed Time: 0:01:41 Time: 0:01:41


Average_labels_per_document in top 10000 class data is 2.4095504116927997
So,Predicting  2 labels using fasttext


100% (437897 of 437897) |#################| Elapsed Time: 0:00:02 Time: 0:00:02


For top 10000 class (precision,recall,fscore) using macro average: (0.25035169296143833, 0.22162339117730662, 0.21473362514162422, None)


In [12]:
obj.fastontop(100000)

100% (2365436 of 2365436) |###############| Elapsed Time: 0:02:40 Time: 0:02:40


Average_labels_per_document in top 100000 class data is 2.982779185253712
So,Predicting  3 labels using fasttext


100% (666171 of 666171) |#################| Elapsed Time: 0:00:04 Time: 0:00:04


For top 100000 class (precision,recall,fscore) using macro average: (0.11915087717637227, 0.12337790169886194, 0.10806798425587555, None)


/home/student1/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/student1/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
